In [1]:
import scvelo as scv
import scanpy as sc
import numpy as np
import pandas as pd
import anndata as ad
import matplotlib.pyplot as plt
import cellrank as cr
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

import psutil

mem = psutil.virtual_memory()
print(f"Total memory: {mem.total / 1e9:.2f} GB")
print(f"Available memory: {mem.available / 1e9:.2f} GB")
print(f"Used memory: {mem.used / 1e9:.2f} GB")
print(f"Memory usage: {mem.percent}%")
# print(globals().keys())

import gc
gc.collect()

Total memory: 540.67 GB
Available memory: 506.57 GB
Used memory: 30.79 GB
Memory usage: 6.3%


40

In [2]:
import sys

def get_vars(scope):
    return {k: v for k, v in scope.items() if not k.startswith('__') and not callable(v)}

def sizeof_fmt(num, suffix='B'):
    for unit in ['','K','M','G','T']:
        if abs(num) < 1024:
            return f"{num:.2f} {unit}{suffix}"
        num /= 1024
    return f"{num:.2f} P{suffix}"

all_vars = get_vars(globals())
var_sizes = {k: sys.getsizeof(v) for k, v in all_vars.items()}
for name, size in sorted(var_sizes.items(), key=lambda x: -x[1]):
    print(f"{name:<30} {sizeof_fmt(size)}")

_i2                            577.00 B
_i                             566.00 B
_i1                            566.00 B
_oh                            232.00 B
Out                            232.00 B
mem                            128.00 B
_ih                            120.00 B
In                             120.00 B
scv                            72.00 B
sc                             72.00 B
np                             72.00 B
pd                             72.00 B
ad                             72.00 B
plt                            72.00 B
cr                             72.00 B
sns                            72.00 B
psutil                         72.00 B
gc                             72.00 B
sys                            72.00 B
_dh                            64.00 B
_ii                            49.00 B
_iii                           49.00 B
_                              28.00 B
_1                             28.00 B


In [5]:
org_merge_day20_WT = org_merge_day20[org_merge_day20.obs["Genotype"] == "WT"].copy()
org_merge_day20_KO = org_merge_day20[org_merge_day20.obs["Genotype"] == "KO"].copy()

In [2]:
# org_merge_KO_changed = sc.read("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_KO_changed.h5ad")
# org_merge_WT_changed = sc.read("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_WT_changed.h5ad")
# org_merge_changed = sc.read("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_changed.h5ad")
# org_merge_KO_changed = sc.read("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_KO_changed_post_velocity.h5ad")
org_merge_day20 = sc.read("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250702_seurat_no_day_60/org_merge_day20_changed.h5ad")


In [4]:
print(org_merge_day20)
scv.tl.latent_time(org_merge_day20)
scv.tl.velocity_confidence(org_merge_day20)

AnnData object with n_obs × n_vars = 12312 × 16203
    obs: 'orig.ident', 'nCount_spliced', 'nFeature_spliced', 'nCount_unspliced', 'nFeature_unspliced', 'nCount_ambiguous', 'nFeature_ambiguous', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'RNA_snn_res.0.8', 'seurat_clusters', 'Sample_ID', 'Genotype', 'Day', 'HNOCA_annot_level_1_pruned', 'HNOCA_annot_level_2_pruned', 'unintegrated_clusters', 'css_cluster', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts'
    var: 'gene_names', 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable', 'fit_r2', 'fit_alpha', 'fit_beta', 'fit_gamma', 'fit_t_', 'fit_scaling', 'fit_std_u', 'fit_std_s', 'fit_likelihood', 'fit_u0', 'fit_s0', 'fit_pval_steady', 'fit_steady_u', 'fit_steady_s', 'fit_variance', 'fit_alignment_scaling'
    uns: 'log1p', 'neighbors', 'recover_dynamics'
    obsm: 'X_css', 'X_pca', 'X_umap.css.sigPCs', 'X_umap.unintegrated'
    varm: 'loss'
    layers: 'Ms', 'Mu', 'ambiguous', 'fit_

In [7]:
import pandas as pd

# mapping of cell types you want to keep
celltype_map = {
    "Neuroepithelium": "Neuroepithelium",
    "Dorsal Telencephalic IP": "Dorsal Telencephalic IP",
    "Dorsal Telencephalic Neuron": "Dorsal Telencephalic Neuron",
    "Ventral Telencephalic NPC": "Ventral Telencephalic NPC",
    "Ventral Telencephalic Neuron": "Ventral Telencephalic Neuron",
    "Non-telencephalic NPC": "Non-telencephalic NPC",
    "Non-telencephalic Neuron": "Non-telencephalic Neuron"
}

# create new obs column
org_merge_day20.obs["HNOCA_annot_level_2_pruned_simplified"] = (
    org_merge_day20.obs["HNOCA_annot_level_2_pruned"]  # your existing 14-celltype annotation
    .map(celltype_map)                       # map to simplified groups
    .fillna("Others")                        # everything else → Others
)


In [6]:
print(org_merge_day20)

AnnData object with n_obs × n_vars = 12312 × 16203
    obs: 'orig.ident', 'nCount_spliced', 'nFeature_spliced', 'nCount_unspliced', 'nFeature_unspliced', 'nCount_ambiguous', 'nFeature_ambiguous', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'RNA_snn_res.0.8', 'seurat_clusters', 'Sample_ID', 'Genotype', 'Day', 'HNOCA_annot_level_1_pruned', 'HNOCA_annot_level_2_pruned', 'unintegrated_clusters', 'css_cluster', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts', 'HNOCA_annot_level_2_pruned_simplified'
    var: 'gene_names', 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable', 'fit_r2', 'fit_alpha', 'fit_beta', 'fit_gamma', 'fit_t_', 'fit_scaling', 'fit_std_u', 'fit_std_s', 'fit_likelihood', 'fit_u0', 'fit_s0', 'fit_pval_steady', 'fit_steady_u', 'fit_steady_s', 'fit_variance', 'fit_alignment_scaling'
    uns: 'log1p', 'neighbors', 'recover_dynamics'
    obsm: 'X_css', 'X_pca', 'X_umap.css.sigPCs', 'X_umap.unintegrated'
    varm: 'loss'
 

In [8]:
scv.tl.latent_time(org_merge_day20)
scv.tl.velocity_confidence(org_merge_day20)

computing terminal states
    identified 4 regions of root cells and 5 regions of end points .
    finished (0:00:01) --> added
    'root_cells', root cells of Markov diffusion process (adata.obs)
    'end_points', end points of Markov diffusion process (adata.obs)
computing latent time using root_cells as prior
    finished (0:00:06) --> added 
    'latent_time', shared time (adata.obs)
--> added 'velocity_length' (adata.obs)
--> added 'velocity_confidence' (adata.obs)
--> added 'velocity_confidence_transition' (adata.obs)


In [9]:
from matplotlib.backends.backend_pdf import PdfPages
import scvelo as scv
import matplotlib.pyplot as plt

pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250702_seurat_no_day_60/velocity_stream_org_merge_day20_HNOCA_annot_level_2_pruned_filtered_simplified.pdf"

r_palette = {
    "Neuroepithelium": "#f44336",
    "Dorsal Telencephalic IP": "#8bc34a",
    "Dorsal Telencephalic Neuron": "#009688",
    "Ventral Telencephalic NPC": "#1565c0",
    "Ventral Telencephalic Neuron": "#ffc107",
    "Non-telencephalic NPC": "#ff9800",
    "Non-telencephalic Neuron": "#448aff",
    "Others": "#ad1457"
}

# Define subsets to plot
subsets = {
    "All cells": org_merge_day20,
    "WT cells": org_merge_day20[org_merge_day20.obs['Genotype'] == 'WT'],
    "KO cells": org_merge_day20[org_merge_day20.obs['Genotype'] == 'KO']
}

with PdfPages(pdf_path) as pdf:
    for label, adata in subsets.items():
        print(f"Plotting {label}...")

        # --------------------------
        # Velocity embedding grid
        # --------------------------
        # scv.pl.velocity_embedding_grid(
        #     adata,
        #     basis='umap.css.sigPCs',
        #     color='HNOCA_annot_level_2_pruned_simplified',
        #     palette=r_palette,
        #     size=50,
        #     scale=0.3,
        #     alpha=0.5,
        #     legend_loc='lower left',
        #     linewidth=1.2,
        #     legend_fontsize=5,
        #     show=False
        # )
        # fig = plt.gcf()
        # fig.set_size_inches(11, 9)
        # fig.suptitle(f"Velocity embedding grid - {label}", fontsize=14)
        # pdf.savefig(fig)
        # plt.close()

        # --------------------------
        # Scatter plot (latent time)
        # --------------------------
        scv.pl.scatter(
            adata,
            basis='umap.css.sigPCs',
            color='latent_time',  # latent time coloring
            color_map='gnuplot',
            size=50,
            alpha=0.8,
            legend_loc='lower left',
            legend_fontsize=5,
            show=False
        )
        fig = plt.gcf()
        fig.set_size_inches(11, 9)
        fig.suptitle(f"Latent time scatter - {label}", fontsize=14)
        pdf.savefig(fig)
        plt.close()

print(f"PDF saved to {pdf_path}")


Plotting All cells...
Plotting WT cells...
Plotting KO cells...
PDF saved to /srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250702_seurat_no_day_60/velocity_stream_org_merge_day20_HNOCA_annot_level_2_pruned_filtered_simplified.pdf


In [4]:
# List the genes you want to check
genes_of_interest = ["GSX2", "ARX", "DLX1", "DLX2"]  # replace with your genes

# Step 1: Find indices of the genes in adata.var
gene_indices = [org_merge_day20_KO.var.index[org_merge_day20_KO.var["gene_names"] == g][0] for g in genes_of_interest]
print(gene_indices)
# Step 2: Extract transcription (alpha), splicing (beta), degradation (gamma)
# These are stored in adata.var
transcription = org_merge_day20_KO.var.loc[gene_indices, "fit_alpha"]
splicing      = org_merge_day20_KO.var.loc[gene_indices, "fit_beta"]
degradation   = org_merge_day20_KO.var.loc[gene_indices, "fit_gamma"]

# Step 3: Combine into a single table for easier viewing
import pandas as pd

df = pd.DataFrame({
    "Gene": genes_of_interest,
    "Transcription (alpha)": transcription.values,
    "Splicing (beta)": splicing.values,
    "Degradation (gamma)": degradation.values
})

print(df)

gene_indices = [org_merge_day20_WT.var.index[org_merge_day20_WT.var["gene_names"] == g][0] for g in genes_of_interest]
print(gene_indices)
# Step 2: Extract transcription (alpha), splicing (beta), degradation (gamma)
# These are stored in adata.var
transcription = org_merge_day20_WT.var.loc[gene_indices, "fit_alpha"]
splicing      = org_merge_day20_WT.var.loc[gene_indices, "fit_beta"]
degradation   = org_merge_day20_WT.var.loc[gene_indices, "fit_gamma"]

# Step 3: Combine into a single table for easier viewing
import pandas as pd

df1 = pd.DataFrame({
    "Gene": genes_of_interest,
    "Transcription (alpha)": transcription.values,
    "Splicing (beta)": splicing.values,
    "Degradation (gamma)": degradation.values
})

print(df1)




NameError: name 'org_merge_day20_KO' is not defined

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages

# Combined dataframe (as you already have)
df_compare = pd.DataFrame({
    "Gene": genes_of_interest,
    "Transcription_WT": df1["Transcription (alpha)"].values,
    "Transcription_KO": df["Transcription (alpha)"].values,
    "Splicing_WT": df1["Splicing (beta)"].values,
    "Splicing_KO": df["Splicing (beta)"].values,
    "Degradation_WT": df1["Degradation (gamma)"].values,
    "Degradation_KO": df["Degradation (gamma)"].values
})

# Add splicing-to-degradation ratios
df_compare["SpliceDegRatio_WT"] = df_compare["Splicing_WT"] / df_compare["Degradation_WT"]
df_compare["SpliceDegRatio_KO"] = df_compare["Splicing_KO"] / df_compare["Degradation_KO"]

# Define colors
colors = {"WT": "#6aa84f", "KO": "#f1c232"}

# Output path
pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250702_seurat_no_day_60/ventral_marker_transcription_splicing.pdf"

# Replace "Degradation" with "Splice/Degradation ratio"
rates = ["Transcription", "Splicing", "Splice/Degradation ratio"]

with PdfPages(pdf_path) as pdf:
    for rate, wt_col, ko_col in zip(
        rates, 
        ["Transcription_WT", "Splicing_WT", "SpliceDegRatio_WT"], 
        ["Transcription_KO", "Splicing_KO", "SpliceDegRatio_KO"]
    ):
        # Log-transform values (handle zeros safely)
        df_plot = df_compare.copy()
        df_plot[[wt_col, ko_col]] = df_plot[[wt_col, ko_col]].applymap(lambda x: np.log1p(x))
        
        # Create figure
        fig, ax = plt.subplots(figsize=(10,6))
        df_plot.plot(
            x="Gene",
            y=[wt_col, ko_col],
            kind="bar",
            ax=ax,
            color=[colors["WT"], colors["KO"]],
            width=0.7
        )
        ax.set_ylabel("log(1 + value)")
        ax.set_title(f"{rate} (log scale)")
        ax.legend(["WT", "KO"], title="Genotype")
        
        # Save page
        pdf.savefig(fig)
        plt.close(fig)

print(f"Saved PDF with 3 plots to: {pdf_path}")


Saved PDF with 3 plots to: /srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250702_seurat_no_day_60/ventral_marker_transcription_splicing.pdf


In [39]:
gene = "GSX2"

# Step 1: find the column index of the gene
gene_idx = org_merge_day20_WT.var.index[org_merge_day20_WT.var["gene_names"] == gene][0]
print(gene_idx)

# Step 2: extract the spliced and unspliced layers
spliced_layer = org_merge_day20_WT.layers["spliced"][:, 13346]
unspliced_layer = org_merge_day20_WT.layers["unspliced"][:, 13346]

# Step 3: count cells with non-zero expression
n_spliced_nonzero = (spliced_layer > 0).sum()
n_unspliced_nonzero = (unspliced_layer > 0).sum()

print(f"Cells with non-zero spliced {gene}: {n_spliced_nonzero}")
print(f"Cells with non-zero unspliced {gene}: {n_unspliced_nonzero}")

42724
Cells with non-zero spliced GSX2: 2
Cells with non-zero unspliced GSX2: 6


In [14]:
gene_name_to_find = "SMO"  # replace with your gene name
index_list = org_merge_day20.var.index[org_merge_day20.var['gene_names'] == gene_name_to_find]


pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250702_seurat_no_day_60/GLI1_expression.pdf"

with PdfPages(pdf_path) as pdf:

    scv.pl.scatter(org_merge_day20, color=index_list[0], basis="X_umap.css.sigPCs", show=False, cmap="plasma_r")
    
    fig = plt.gcf()
    fig.set_size_inches(10, 7)  
    pdf.savefig(fig)           
    plt.close()


In [21]:
import pandas as pd
from scipy.stats import ttest_ind, mannwhitneyu

genes = ["GLI1", "GLI2", "GLI3", "PTCH1", "PTCH2", "SMO", "SHH"]
results = []

for g in genes:
    idx_list = org_merge_day20.var.index[org_merge_day20.var['gene_names'] == g]
    if len(idx_list) == 0:
        continue  # skip if gene not present
    gene_idx = idx_list[0]

    # expression values
    expr = org_merge_day20[:, gene_idx].X.toarray().flatten()

    # split by genotype
    wt = expr[org_merge_day20.obs['Genotype'] == "WT"]
    ko = expr[org_merge_day20.obs['Genotype'] == "KO"]

    # stats
    stat_t, p_t = ttest_ind(wt, ko, equal_var=False)
    stat_w, p_w = mannwhitneyu(wt, ko, alternative="two-sided")

    results.append([g, wt.mean(), ko.mean(), ko.mean() - wt.mean(), p_t, p_w])

df_results = pd.DataFrame(results, columns=["Gene", "Mean_WT", "Mean_KO", "Difference", "p_ttest", "p_wilcoxon"])
df_results


,Gene,Mean_WT,Mean_KO,Difference,p_ttest,p_wilcoxon
0,GLI1,0.000218,0.001697,0.001479,9.526335e-04,1.624337e-04
1,GLI2,0.059664,0.091142,0.031478,1.170339e-13,3.088556e-23
2,GLI3,0.269591,0.420510,0.150919,2.029713e-68,9.824834e-70
3,PTCH1,0.045344,0.105854,0.060510,9.541603e-46,3.583915e-65
4,PTCH2,0.005214,0.006083,0.000869,4.491029e-01,5.142614e-03
5,SMO,0.102558,0.146221,0.043663,7.363529e-17,6.109867e-27
6,SHH,0.000130,0.000409,0.000279,2.516991e-01,1.906771e-01


In [57]:
matrix = org_merge_KO_changed.X
dense_matrix = matrix.toarray()
df = pd.DataFrame(dense_matrix, index=org_merge_KO_changed.obs_names, columns=org_merge_KO_changed.var_names)
df.to_csv("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_KO_changed_anndata_matrix.csv")

In [5]:
org_merge_WT_changed.var.to_csv("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_WT_changed_var.csv")
org_merge_WT_changed.obs.to_csv("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_WT_changed_obs.csv")
velocity_df = pd.DataFrame(org_merge_WT_changed.layers["velocity"], 
                           index=org_merge_WT_changed.obs_names,
                           columns=org_merge_WT_changed.var_names)

velocity_df.to_csv("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_WT_changed_velocity.csv")
velocity_u_df = pd.DataFrame(org_merge_WT_changed.layers["velocity_u"],
                             index=org_merge_WT_changed.obs_names,
                             columns=org_merge_WT_changed.var_names)

velocity_u_df.to_csv("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_WT_changed_velocity_u.csv")

In [ ]:
org_merge_KO_changed.var.to_csv("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_KO_changed_var.csv")
org_merge_KO_changed.obs.to_csv("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_KO_changed_obs.csv")
velocity_df = pd.DataFrame(org_merge_KO_changed.layers["velocity"], 
                           index=org_merge_KO_changed.obs_names,
                           columns=org_merge_KO_changed.var_names)

velocity_df.to_csv("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_KO_changed_velocity.csv")
velocity_u_df = pd.DataFrame(org_merge_KO_changed.layers["velocity_u"],
                             index=org_merge_KO_changed.obs_names,
                             columns=org_merge_KO_changed.var_names)

velocity_u_df.to_csv("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_KO_changed_velocity_u.csv")

In [ ]:
# scv.pl.proportions(org_merge_day20, groupby="Sample_ID")
# scv.pl.proportions(org_merge_no_day60, groupby="Sample_ID")
# scv.pl.proportions(org_merge_no_day60_WT, groupby="Sample_ID")
scv.pl.proportions(org_merge_changed, groupby="Sample_ID", show=False)
plt.savefig("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/proportions_KO.pdf", bbox_inches='tight')

# initial_size_unspliced = scv.core.get_initial_size(org_merge_WT_changed, layer='unspliced')
# initial_size_spliced = scv.core.get_initial_size(org_merge_WT_changed, layer='spliced')
# initial_size_ambiguous = scv.core.get_initial_size(org_merge_WT_changed, layer='ambiguous')

# total_counts = np.sum([initial_size_unspliced, initial_size_spliced, initial_size_ambiguous], axis=0)

# fraction_unspliced = initial_size_unspliced / total_counts
# mean_fraction_unspliced = fraction_unspliced.mean() 
# print(mean_fraction_unspliced)

# fraction_spliced = initial_size_spliced / total_counts
# mean_fraction_spliced = fraction_spliced.mean()
# print(mean_fraction_spliced)


In [5]:
print(org_merge_KO_changed.var['highly_variable'][0:30])

0      True
12     True
19     True
23     True
24     True
39     True
69     True
83     True
87     True
119    True
128    True
139    True
142    True
149    True
155    True
156    True
161    True
164    True
170    True
191    True
197    True
214    True
220    True
222    True
224    True
249    True
269    True
274    True
278    True
283    True
Name: highly_variable, dtype: bool


In [26]:
org_merge_KO_changed.obs.to_csv("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_KO_changed_obs.csv")
org_merge_KO_changed.var.to_csv("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_KO_changed_var.csv")

velocity = org_merge_KO_changed.layers["velocity"]
velocity_df = pd.DataFrame(
    velocity,
    index=org_merge_KO_changed.obs_names,
    columns=org_merge_KO_changed.var_names
)
velocity_df.to_csv("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_KO_changed_velocity.csv")

velocity_u = org_merge_KO_changed.layers["velocity_u"]
velocity_u_df = pd.DataFrame(
    velocity_u,
    index=org_merge_KO_changed.obs_names,
    columns=org_merge_KO_changed.var_names
)
velocity_u_df.to_csv("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_KO_changed_velocity_u.csv")

In [8]:
import numpy as np

gene_index = 12
# print(org_merge_KO_changed.layers['spliced'])
print(org_merge_KO_changed.layers['Ms'].shape)
print(np.max(org_merge_KO_changed.layers['Mu'][:,1]))
print(np.max(org_merge_KO_changed.layers['Ms'][:,1]))

print(type(org_merge_KO_changed.layers['Ms']))
rows, cols = np.where(org_merge_KO_changed.layers['Ms'] == 4.1331363)
print(rows)
print(cols)

spliced = org_merge_KO_changed.layers['Ms'][:, gene_index].flatten().max()
print(spliced)
unspliced = org_merge_KO_changed.layers['Mu'][:, gene_index].flatten().max()
print(unspliced)
# spliced = np.ravel(org_merge_KO_changed.layers['Ms'][:, gene_index] > 0)
# print(spliced)
# unspliced = np.ravel(org_merge_KO_changed.layers['Mu'][:, gene_index] > 0)
weights = spliced & unspliced
print(np.sum(weights))

# Boolean arrays for cells with counts > 0
nonzero_s = spliced > 0
nonzero_u = unspliced > 0

# Cells where both spliced and unspliced > 0
weights = nonzero_s & nonzero_u

# Check if gene is recoverable (more than 2 such cells)
recoverable = np.sum(weights) > 2

print(f"Gene 192 recoverable? {recoverable}")
print(f"Number of cells with both spliced & unspliced > 0: {np.sum(weights)}")


(55582, 2256)
0.8459427
8.308589
<class 'numpy.ndarray'>
[17174 52137]
[  29 2004]
2.155674
0.4025494


TypeError: ufunc 'bitwise_and' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [25]:
Ms = org_merge_KO_changed.layers['Ms']

# Convert to dense array if sparse
if hasattr(Ms, "toarray"):
    Ms_dense = Ms.toarray()
else:
    Ms_dense = Ms  # already dense

# Convert to DataFrame and assign cell and gene labels
df = pd.DataFrame(
    Ms_dense,
    index=org_merge_KO_changed.obs_names,  # cell barcodes
    columns=org_merge_KO_changed.var_names  # gene names
)

# Save to CSV
df.to_csv("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/Ms_layer.csv")

In [13]:
import sys, scvelo
print(sys.executable)  # Path to the Python interpreter
print(scvelo.__file__)  # Path to the scvelo package
print(scvelo.__version__)

/apps/z_install_tree/linux-rocky8-ivybridge/gcc-12.2.0/python-3.10.8-pmtwsrrmcmrs6olvgx5xhepgh7gl5vro/bin/python3
/home/z5358666/.local/lib/python3.10/site-packages/scvelo/__init__.py
0.3.3


In [7]:
scv.pp.filter_and_normalize(org_merge_day20_KO, min_cells =2, min_cells_u=2)
scv.pp.filter_genes_dispersion(org_merge_day20_KO, subset=False)
sc.pp.neighbors(org_merge_day20_KO, n_neighbors=20, use_rep='X_css')
scv.pp.moments(org_merge_day20_KO, n_pcs=None, n_neighbors=None)

Filtered out 31029 genes that are detected in less than 2 cells (spliced).
Filtered out 7810 genes that are detected in less than 2 cells (unspliced).
Normalized count data: X, spliced, unspliced.
Logarithmized X.


/home/z5358666/.local/lib/python3.10/site-packages/scvelo/preprocessing/utils.py:705: DeprecationWarning: `log1p` is deprecated since scVelo v0.3.0 and will be removed in a future version. Please use `log1p` from `scanpy.pp` instead.
  log1p(adata)


Extracted 2090 highly variable genes.
computing moments based on connectivities
    finished (0:00:10) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


In [9]:
var_names = ['57810', '33474', '31437']
scv.tl.recover_dynamics(org_merge_day20_KO, var_names=var_names)

recovering dynamics (using 1/128 cores)
    finished (0:00:06) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)

outputs model fit of gene: 31437


In [7]:
scv.tl.velocity(org_merge_day20, mode='dynamical')
scv.tl.velocity_graph(org_merge_day20)

computing velocities
    finished (0:00:15) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
computing velocity graph (using 1/44 cores)
or disable the progress bar using `show_progress_bar=False`.
    finished (0:00:16) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)


In [17]:
org_merge_day20.write("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250702_seurat_no_day_60/org_merge_day20.h5ad")
# org_merge_day60.write("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250702_seurat_no_day_60/org_merge_no_day60.h5ad")
# org_merge_day60_WT.write("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250702_seurat_no_day_60/org_merge_no_day60_WT.h5ad")


In [7]:
pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_changed_velocity.pdf"

r_palette = {
    "Dorsal Telencephalic IP": "#E41A1C",
    "Dorsal Telencephalic NPC": "#377EB8",
    "MC": "#4DAF4A",
    "NA": "#984EA3",
    "NC Derivatives": "#FF7F00",
    "Neuroepithelium": "#A65628",
    "Non-telencephalic NPC": "#F781BF",
    "Non-telencephalic Neuron": "#999999",
    "PSC": "#66C2A5",
    "Ventral Telencephalic NPC": "#FFD92F",
    "Ventral Telencephalic Neuron": "#7F00FF",
    "Astrocyte": "#4B0082",
    "CP": "#056517",
    "Dorsal Telencephalic Neuron": "#8DD3C7",
    "Glioblast": "#FFA33F",
    "OPC": "#80B1D3"
}

with PdfPages(pdf_path) as pdf:

    scv.pl.velocity_embedding_grid(
        org_merge_changed,
        basis='umap.css.sigPCs',
        color='HNOCA_annot_level_2_pruned',
        palette=r_palette,
        size=50,
        scale=0.3,               
        alpha=0.7,
        legend_loc='lower left',
        legend_fontsize=5,
        show=False
    )
    
    fig = plt.gcf()
    fig.set_size_inches(10, 8)  
    pdf.savefig(fig)           
    plt.close()

computing velocity embedding
    finished (0:00:15) --> added
    'velocity_umap.css.sigPCs', embedded velocity vectors (adata.obsm)


/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:68: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  return isinstance(c, str) and c in data.obs.keys() and cat(data.obs[c])
/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:68: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  return isinstance(c, str) and c in data.obs.keys() and cat(data.obs[c])
/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:65: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  return cat(data)  # if data is categorical/array
/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:68: DeprecationWarning: is_categorical_dtype

In [8]:
pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_changed_latent_time_velocity_confidence.pdf"

scv.tl.latent_time(org_merge_changed)
scv.tl.velocity_confidence(org_merge_changed)

with PdfPages(pdf_path) as pdf:
    scv.pl.scatter(
        org_merge_changed,  
        color="latent_time", 
        color_map='gnuplot', 
        basis="X_umap.css.sigPCs",
        size=80,
        show=False
    )
    fig = plt.gcf()
    fig.set_size_inches(10, 7)
    pdf.savefig(fig)
    plt.close()
    
    scv.pl.scatter(
        org_merge_changed, 
        color="velocity_length", 
        color_map="gnuplot", 
        basis="X_umap.css.sigPCs",
        size=80,
        show=False
    )
    fig = plt.gcf()
    fig.set_size_inches(10, 7)
    pdf.savefig(fig)
    plt.close()

    scv.pl.scatter(
        org_merge_changed, 
        color="velocity_confidence", 
        color_map="gnuplot", 
        basis="X_umap.css.sigPCs",
        size=80,
        show=False
    )

    fig = plt.gcf()
    fig.set_size_inches(10, 7)  
    pdf.savefig(fig)           
    plt.close()

computing terminal states
    identified 10 regions of root cells and 7 regions of end points .
    finished (0:01:38) --> added
    'root_cells', root cells of Markov diffusion process (adata.obs)
    'end_points', end points of Markov diffusion process (adata.obs)
computing latent time using root_cells as prior
    finished (0:00:36) --> added 
    'latent_time', shared time (adata.obs)
--> added 'velocity_length' (adata.obs)
--> added 'velocity_confidence' (adata.obs)
--> added 'velocity_confidence_transition' (adata.obs)


/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:68: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  return isinstance(c, str) and c in data.obs.keys() and cat(data.obs[c])
/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:68: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  return isinstance(c, str) and c in data.obs.keys() and cat(data.obs[c])
/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:68: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  return isinstance(c, str) and c in data.obs.keys() and cat(data.obs[c])
/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:68: DeprecationWarnin

In [28]:
top_genes = org_merge_WT_changed.var['fit_likelihood'].sort_values(ascending=False).index[:5]
print(org_merge_WT_changed.var['fit_likelihood'].sort_values(ascending=False)[0:5])
print(org_merge_WT_changed)  

# indices = ['5770', '11156', '4798', '4757', '10175'] 
# indices = ['13654', '12735', '10175', '15386', '11682']
indices = ['17792']


for index in indices:
    gene_name = org_merge_WT_changed.var['gene_names'][index]

    pdf_path = f"/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/{gene_name}_WT.pdf"
    print(pdf_path)
    
    with PdfPages(pdf_path) as pdf:

        scv.pl.scatter(
            org_merge_WT_changed,
            basis="X_umap.css.sigPCs",
            var_names = str(index),
            color = 'latent_time',
            size=30,
            show=False,
            legend_loc='upper left'
        )

        fig = plt.gcf()
        fig.set_size_inches(10, 7)
        pdf.savefig(fig)
        plt.close() 

        scv.pl.scatter(
            org_merge_WT_changed,
            basis="X_umap.css.sigPCs",
            var_names = str(index),
            color = 'HNOCA_annot_level_2_pruned',
            size=30,
            show=False,
            legend_loc='upper left'
        )

        fig = plt.gcf()
        fig.set_size_inches(10, 7)
        pdf.savefig(fig)
        plt.close()
        

        scv.pl.velocity(
            org_merge_WT_changed,
            basis="X_umap.css.sigPCs",
            var_names = str(index),
            color ='HNOCA_annot_level_2_pruned',
            size=30,
            show=False,
            legend_loc='upper left'
        )

        fig = plt.gcf()
        fig.set_size_inches(30, 7)
        pdf.savefig(fig)
        plt.close() 

5770     0.474149
10175    0.407594
11156    0.395738
4798     0.385830
4757     0.378705
Name: fit_likelihood, dtype: float64
AnnData object with n_obs × n_vars = 39208 × 1937
    obs: 'nCount_ambiguous', 'nFeature_ambiguous', 'nCount_spliced', 'nFeature_spliced', 'nCount_unspliced', 'nFeature_unspliced', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'percent.rps', 'percent.rpl', 'RNA_snn_res.0.1', 'seurat_clusters', 'pANN_0.25_0.3_354', 'doublet_finder', 'pANN_0.25_0.03_363', 'pANN_0.25_0.005_663', 'pANN_0.25_0.17_698', 'pANN_0.25_0.005_717', 'pANN_0.25_0.01_706', 'pANN_0.25_0.005_755', 'pANN_0.25_0.001_663', 'pANN_0.25_0.26_795', 'Sample_ID', 'unintegrated_clusters', 'clusters_CSS_allEmbeddingDims_Point1', 'clusters_CSS_SigPs_Point1', 'clusters_Harmony_Point1', 'clusters_CSS_allEmbeddingDims_Point2', 'clusters_CSS_SigPs_Point2', 'clusters_Harmony_Point2', 'clusters_CSS_allEmbeddingDims_Point5', 'clusters_CSS_SigPs_Point5', 'clusters_Harmony_Point5', 'clusters_CSS_SigPs_0

/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/scatter.py:656: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  smp = ax.scatter(


In [9]:
# org_merge_WT_changed.write("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_WT_changed_post_velocity.h5ad")
# org_merge_KO_changed.write("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_KO_changed_post_velocity.h5ad")
org_merge_day20.write("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_day20_changed.h5ad")

In [23]:
scv.tl.differential_kinetic_test(org_merge_KO_changed, groupby="HNOCA_annot_level_2_pruned")

testing for differential kinetics
    finished (0:07:06) --> added 
    'fit_diff_kinetics', clusters displaying differential kinetics (adata.var)
    'fit_pvals_kinetics', p-values of differential kinetics (adata.var)


In [24]:
scv.tl.velocity(org_merge_KO_changed, mode='dynamical', diff_kinetics=True)
scv.tl.velocity_graph(org_merge_KO_changed)

computing velocities
    finished (0:01:11) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
computing velocity graph (using 1/64 cores)
    finished (0:00:46) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)


In [ ]:
### BUFFER ####################

In [ ]:
print(org_merge_day20.var['gene_names'])
print('FOXG1' in org_merge_day20.var['gene_names'].values)
matching_row = org_merge_day20.var[org_merge_day20.var['gene_names'] == 'FOXG1'] #16946
print(matching_row)

pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250702_seurat_no_day_60/FOXG1_day20.pdf"


with PdfPages(pdf_path) as pdf:
    scv.pl.velocity(org_merge_day20, var_names=['16946'], basis='X_umap.css.sigPCs', color="HNOCA_annot_level_2_pruned", show=False, size=10)
    fig = plt.gcf()
    fig.set_size_inches(30, 7)  
    pdf.savefig(fig)           
    plt.close()


In [ ]:
# print(org_merge_day20)
org_merge_day20_WT = org_merge_day20[org_merge_day20.obs["Genotype"] == "WT"].copy()
org_merge_day20_KO = org_merge_day20[org_merge_day20.obs["Genotype"] == "KO"].copy()

In [ ]:
scv.pp.filter_and_normalize(org_merge_no_day60_KO)
sc.pp.neighbors(org_merge_no_day60_KO, n_neighbors=20, use_rep='X_pca')
scv.pp.moments(org_merge_no_day60_KO, n_pcs = 20, n_neighbors = 20)
scv.tl.recover_dynamics(org_merge_no_day60_KO)
scv.tl.velocity(org_merge_no_day60_KO, mode='dynamical')
scv.tl.velocity_graph(org_merge_no_day60_KO)

In [ ]:
pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250702_seurat_no_day_60/org_merge_no_day60_UMAP_HNOCA_annot_level_2_pruned_WT.pdf"

r_palette = {
    "Dorsal Telencephalic IP": "#E41A1C",
    "Dorsal Telencephalic NPC": "#377EB8",
    "MC": "#4DAF4A",
    "NA": "#984EA3",
    "NC Derivatives": "#FF7F00",
    "Neuroepithelium": "#A65628",
    "Non-telencephalic NPC": "#F781BF",
    "Non-telencephalic Neuron": "#999999",
    "PSC": "#66C2A5",
    "Ventral Telencephalic NPC": "#FFD92F",
    "Ventral Telencephalic Neuron": "#7F00FF",
    "Astrocyte": "#A6D854",
    "CP": "#B3B3B3",
    "Dorsal Telencephalic Neuron": "#8DD3C7",
    "Glioblast": "#FB8072",
    "OPC": "#80B1D3"
}



In [ ]:
import numpy as np

pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250702_seurat_no_day_60/velocity_stream_org_merge_day20_HNOCA_annot_level_2_pruned_filtered.pdf"

# Define your categories of interest
target_celltypes = {
    "Neuroepithelium": "#377EB8",
    "Ventral Telencephalic Neuron": "#7B3294",
    "Non-telencephalic Neuron": "#FDAE6B",
    "PSC": "#E41A1C",    
    "Other": "#4DAF4A"     
}

# Create a new annotation column
org_merge_day20.obs['celltype_grouped'] = org_merge_day20.obs['HNOCA_annot_level_2_pruned'].copy()
org_merge_day20.obs['celltype_grouped'] = org_merge_day20.obs['celltype_grouped'].apply(
    lambda x: x if x in target_celltypes else 'Other'
)

# Plot
with PdfPages(pdf_path) as pdf:
    scv.pl.velocity_embedding_grid(
        org_merge_day20,
        basis='umap.css.sigPCs',
        color='celltype_grouped',
        palette=target_celltypes,
        size=50,
        scale=0.3,
        alpha=0.7,
        legend_loc='lower left',
        legend_fontsize=5,
        show=False
    )

    fig = plt.gcf()
    fig.set_size_inches(10, 7)
    pdf.savefig(fig)
    plt.close()


In [ ]:
pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250702_seurat_no_day_60/org_merge_day20_UMAP_HNOCA_annot_level_2_pruned.pdf"

with PdfPages(pdf_path) as pdf:
    scv.pl.scatter(
        org_merge_day20,
        basis='umap.css.sigPCs',
        color='celltype_grouped',
        palette=target_celltypes,
        size=50,
        alpha=0.7,
        legend_loc='lower left',
        legend_fontsize=5,
        show=False
    )

    fig = plt.gcf()
    fig.set_size_inches(10, 7)
    pdf.savefig(fig)
    plt.close()

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250702_seurat_no_day_60/org_merge_day20_UMAP_HNOCA_annot_Genotype.pdf"

# Define genotype colors
genotype_palette = {
    "WT": "#6aa84f",   # Yellow-gold
    "KO": "#f1c232"    # Green
}

# (Optional) Clean the values if needed
org_merge_day20.obs['genotype_grouped'] = org_merge_day20.obs['Genotype'].apply(
    lambda x: x if x in genotype_palette else 'Other'
)

with PdfPages(pdf_path) as pdf:
    scv.pl.scatter(
        org_merge_day20,
        basis='umap.css.sigPCs',
        color='genotype_grouped',     # Set the correct column here
        palette=genotype_palette,     # Use the defined palette
        size=10,
        alpha=0.7,
        legend_loc='lower left',
        legend_fontsize=5,
        show=False
    )

    fig = plt.gcf()
    fig.set_size_inches(10, 7)
    pdf.savefig(fig)
    plt.close()


In [ ]:
org_merge_no_day60_KO.obs["highlight"] = org_merge_no_day60_KO.obs["HNOCA_annot_level_1_pruned"] == "PSC"
pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250702_seurat_no_day_60/org_merge_no_day60_UMAP_HNOCA_annot_level_2_pruned_KO_PSC_abundance.pdf"


with PdfPages(pdf_path) as pdf:

    scv.pl.scatter(org_merge_no_day60_KO, color="highlight", palette=["lightgrey", "red"], title="PSC", basis="X_umap.css.sigPCs", show=False)
    
    fig = plt.gcf()
    fig.set_size_inches(10, 7)  
    pdf.savefig(fig)           
    plt.close()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250702_seurat_no_day_60/org_merge_day20_bar_graph.pdf"

cell_type_counts = org_merge_day20.obs['HNOCA_annot_level_2_pruned'].value_counts(normalize=True)

# Save to PDF
with PdfPages(pdf_path) as pdf:
    fig, ax = plt.subplots(figsize=(10, 7))

    # Plot bar with log scale
    cell_type_counts.plot(kind='bar', color='steelblue', ax=ax)
    ax.set_yscale('log')
    ax.set_ylabel('Fraction of cells (log scale)')
    ax.set_xlabel('Cell type')
    ax.set_title('Cell type distribution in D20 organoid')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    # Save and close
    pdf.savefig(fig)
    plt.close(fig)



In [ ]:
###### ONLY USE FOR DAY 20

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import LogLocator, ScalarFormatter

pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250702_seurat_no_day_60/org_merge_day20_WT_bar_graph_combined.pdf"

# Get raw counts
cell_type_series = org_merge_day20_WT.obs['HNOCA_annot_level_2_pruned']

# Define your categories of interest
main_categories = [
    "Neuroepithelium",
    "Non-telencephalic Neuron",
    "Ventral Telencephalic Neuron",

    # "Dorsal Telencephalic Neuron",
]

# Count matching categories
grouped_counts = {
    category: (cell_type_series == category).sum()
    for category in main_categories
}

# Count all other cell types
grouped_counts["Other"] = cell_type_series[~cell_type_series.isin(main_categories)].count()

# Normalize to get fractions
total_cells = len(cell_type_series)
grouped_fractions = {k: v / total_cells for k, v in grouped_counts.items()}

# --- Plot log-scale bar chart ---
with PdfPages(pdf_path) as pdf:
    fig, ax = plt.subplots(figsize=(10, 7))
    
    labels = list(grouped_fractions.keys())
    values = list(grouped_fractions.values())
    
    ax.bar(labels, values, color='steelblue')
    ax.set_yscale('log')
    ax.set_ylabel('Fraction of cells (log scale)')
    ax.set_xlabel('Cell type group')
    ax.set_title('Cell type distribution in D20 organoid (log-scale bar chart)')
    plt.xticks(rotation=45, ha='right')

    # Optional: show more y-ticks
    ax.yaxis.set_major_locator(LogLocator(base=10.0, subs=(1.0,), numticks=10))
    ax.yaxis.set_minor_locator(LogLocator(base=10.0, subs=range(2, 10), numticks=100))
    ax.yaxis.set_minor_formatter(ScalarFormatter())
    ax.yaxis.set_minor_formatter(plt.NullFormatter())  # Hide minor tick labels

    plt.tight_layout()
    pdf.savefig(fig)
    plt.close(fig)


In [24]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import LogLocator, ScalarFormatter
import numpy as np

pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250702_seurat_no_day_60/org_merge_day20_WT_KO_bar_graph_combined.pdf"

# Define categories of interest
main_categories = [
    "Neuroepithelium",
    "Non-telencephalic Neuron",
    "Ventral Telencephalic Neuron"
    # "PSC",
    # "Dorsal Telencephalic Neuron",
]

def compute_grouped_fractions(cell_type_series, main_categories):
    grouped_counts = {
        category: (cell_type_series == category).sum()
        for category in main_categories
    }
    grouped_counts["Other"] = cell_type_series[~cell_type_series.isin(main_categories)].count()
    
    total_cells = len(cell_type_series)
    return {k: v / total_cells for k, v in grouped_counts.items()}

# Compute fractions
fractions_WT = compute_grouped_fractions(org_merge_day20_WT.obs['HNOCA_annot_level_2_pruned'], main_categories)
fractions_KO = compute_grouped_fractions(org_merge_day20_KO.obs['HNOCA_annot_level_2_pruned'], main_categories)

# Prepare plot
labels = list(fractions_WT.keys())  # Same keys in both dictionaries
x = np.arange(len(labels))          # the label locations
width = 0.35                        # width of the bars

with PdfPages(pdf_path) as pdf:
    fig, ax = plt.subplots(figsize=(10, 7))

    # Bar plots
    ax.bar(x - width/2, [fractions_WT[k] for k in labels], width, label='WT', color='#6aa84f')
    ax.bar(x + width/2, [fractions_KO[k] for k in labels], width, label='KO', color='#f1c232')

    ax.set_yscale('log')
    ax.set_ylabel('Proportion (log scale)')
    ax.set_xlabel('Cell type')
    ax.set_title('Cell type distribution in D20 organoids (WT vs KO)')
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=45, ha='right')

    # Log-scale ticks
    ax.yaxis.set_major_locator(LogLocator(base=10.0, subs=(1.0,), numticks=10))
    ax.yaxis.set_minor_locator(LogLocator(base=10.0, subs=range(2, 10), numticks=100))
    ax.yaxis.set_minor_formatter(plt.NullFormatter())

    ax.legend()
    plt.tight_layout()
    pdf.savefig(fig)
    plt.close(fig)


NameError: name 'org_merge_day20_WT' is not defined

In [ ]:
### USE FOR DAY 100
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250702_seurat_no_day_60/org_merge_no_day60_KO_bar_graph.pdf"

cell_type_counts = org_merge_no_day60_KO.obs['HNOCA_annot_level_2_pruned'].value_counts(normalize=True)

# Save to PDF
with PdfPages(pdf_path) as pdf:
    fig, ax = plt.subplots(figsize=(10, 7))

    # Plot bar with log scale
    cell_type_counts.plot(kind='bar', color='steelblue', ax=ax)
    ax.set_yscale('log')
    ax.set_ylabel('Fraction of cells (log scale)')
    ax.set_xlabel('Cell type')
    ax.set_title('Cell type distribution in D20 organoid')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    # Save and close
    pdf.savefig(fig)
    plt.close(fig)


In [23]:
import scvelo as scv
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Define PDF path
pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_day20_changed_latent_time_velocity_confidence.pdf"

# Compute latent time and velocity confidence
scv.tl.latent_time(org_merge_day20)
scv.tl.velocity_confidence(org_merge_day20)

# Save scatter plot to PDF
with PdfPages(pdf_path) as pdf:
    scv.pl.scatter(
        org_merge_day20,  
        color="latent_time", 
        color_map='gnuplot',  # or 'viridis', etc.
        basis="X_umap.css.sigPCs",
        size=80,
        show=False  # prevent immediate display
    )
    fig = plt.gcf()            # get current figure
    fig.set_size_inches(10, 7) # optional resizing
    pdf.savefig(fig)            # save to PDF
    plt.close(fig)              # close figure to free memory

print(f"PDF saved to {pdf_path}")


computing latent time using root_cells as prior
    finished (0:00:07) --> added 
    'latent_time', shared time (adata.obs)
--> added 'velocity_length' (adata.obs)
--> added 'velocity_confidence' (adata.obs)
--> added 'velocity_confidence_transition' (adata.obs)
PDF saved to /srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_day20_changed_latent_time_velocity_confidence.pdf


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Extract data
data = org_merge_changed.obs[['Day', 'latent_time']].copy()
data['Day'] = data['Day'].astype(str)

# Optional: order Days numerically
data['Day'] = data['Day'].astype('category')
data['Day'].cat.set_categories(['20', '60', '100'], ordered=True, inplace=True)

# Plot
plt.figure(figsize=(6, 4))
sns.violinplot(x='Day', y='latent_time', data=data, inner='box', palette='Set2')

plt.xlabel('Day')
plt.ylabel('Latent Time')
plt.title('Latent Time Distribution across Days')

plt.tight_layout()

# ---- Save directly to your desired PDF path ----
pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250702_seurat_no_day_60/velocity_stream_org_merge_day20_HNOCA_annot_level_2_pruned_filtered_simplified.pdf"
plt.savefig(pdf_path, format='pdf', bbox_inches='tight')
plt.close()

print(f"Saved plot to: {pdf_path}")


/scratch/pbs.6997009.kman.restech.unsw.edu.au/ipykernel_2860723/4256905583.py:54: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(celltype_order, rotation=45, ha="right")
/scratch/pbs.6997009.kman.restech.unsw.edu.au/ipykernel_2860723/4256905583.py:54: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(celltype_order, rotation=45, ha="right")
/scratch/pbs.6997009.kman.restech.unsw.edu.au/ipykernel_2860723/4256905583.py:54: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(celltype_order, rotation=45, ha="right")


PDF saved to /srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250909_celltype_proportions_logbar.pdf
